In [1]:
!az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7b658efc-8cee-4580-b989-85db158d4e3c",
    "id": "3979595d-b401-4ff4-b3ef-661b8481c742",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "961897a7-5808-4b80-b503-dbb7be13e7a7"
      }
    ],
    "name": "MV-AZU-001",
    "state": "Enabled",
    "tenantId": "7b658efc-8cee-4580-b989-85db158d4e3c",
    "user": {
      "name": "ben.dixon@multiverse.io",
      "type": "user"
    }
  }
]


In [2]:
import uuid
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Environment
from azure.ai.ml.entities import ManagedOnlineEndpoint
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

/Users/ben.dixon/Repos/azure-experiments/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /Users/ben.dixon/Repos/azure-experiments/config.json


## Register model and get latest version

In [7]:
REGISTERED_MODEL_NAME = "xgboost_model"

xgboost_model = Model(
    path="../deploy/assets/xgboost_model.json",
    type=AssetTypes.CUSTOM_MODEL,
    name=REGISTERED_MODEL_NAME,
    description="Model created from local files.",
)
ml_client.models.create_or_update(xgboost_model)

latest_model_version = max(
        [int(m.version) for m in ml_client.models.list(name=REGISTERED_MODEL_NAME)]
    )
print(f"Latest model version: {latest_model_version}")
latest_model = ml_client.models.get(name=REGISTERED_MODEL_NAME, version=latest_model_version)


Latest model version: 21


## Register environment

In [8]:
env = Environment(
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        conda_file="../azureml_envs/xgboost.yaml",
        name="xgboost-inference-env",
        description="Environment for xgboost inference created from a Docker image plus Conda environment.",
    )
ml_client.environments.create_or_update(env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'xgboost-inference-env', 'description': 'Environment for xgboost inference created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourceGroups/aiplatform-1/providers/Microsoft.MachineLearningServices/workspaces/ai-mlw-dev-01/environments/xgboost-inference-env/versions/22', 'Resource__source_path': '', 'base_path': '/Users/ben.dixon/Repos/azure-experiments/deploy', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x1456d81c0>, 'serialize': <msrest.serialization.Serializer object at 0x1456d87c0>, 'version': '22', 'conda_file': {'channels': ['conda-forge'], 'dependencies

## Create local endpoint 

In [16]:
online_endpoint_name = "xgboost-model-endpoint-" + str(uuid.uuid4())[:8]
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint for the xgboost model and it's great!",
    auth_mode="key",
    tags={
        "training_dataset": "sklearn_iris",
    },
)
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True) # takes approximately 2 minutes.

Creating local endpoint (xgboost-model-endpoint-2e60ced8) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'xgboost-model-endpoint-2e60ced8', 'description': "this is an online endpoint for the xgboost model and it's great!", 'tags': {'training_dataset': 'sklearn_iris'}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/Users/ben.dixon/.azureml/inferencing/xgboost-model-endpoint-2e60ced8'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x1465a9d30>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## Create a local deployment

Local endpoints only support local artifacts, so loading the model locally. 

In [17]:
# model = ml_client.models.get(name=REGISTERED_MODEL_NAME, version=latest_model_version)
local_model = Model(path="../deploy/assets/xgboost_model.json")

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=local_model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
    code_configuration=CodeConfiguration(
        code="../deploy/", 
        scoring_script="score.py"
    ),
    environment=env
)

blue_deployment = ml_client.online_deployments.begin_create_or_update(
        deployment=blue_deployment, local=True
) 

Creating local deployment (xgboost-model-endpoint-2e60ced8 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04
 ---> 948ed3518cc2
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in f55087ccc2a9
 ---> a7d5e2241042
Step 3/6 : WORKDIR /var/azureml-app/
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in 2d3dc5ef7c95
 ---> 95f0c796d118
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> 6a4f7d9457f7
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in 7c12167

### Check the status
Checking to see whether the model was deployed without error 

In [18]:
ml_client.online_endpoints.get(name=online_endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:55005/score', 'openapi_uri': None, 'name': 'xgboost-model-endpoint-2e60ced8', 'description': "this is an online endpoint for the xgboost model and it's great!", 'tags': {'training_dataset': 'sklearn_iris'}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/Users/ben.dixon/Repos/azure-experiments/deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x1465ca1f0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

The sample xgboost request is the first four values from the test set. 

In [19]:
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="assets/xgboost-request.json",
    local=True,
)

'[1, 0, 2, 1]'

In [41]:
!python ../src/train-xgboost.py

(30, 4)
Fitting model
Making predictions
Mean squared error: 0.00
Saving model
Predictions on test set [1, 0, 2, 1]


Success, these two match!

## Deploy your online endpoint to Azure

Change to using registered model. 

In [23]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result() # takes approximately 2 minutes.

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://xgboost-model-endpoint-2e60ced8.ukwest.inference.ml.azure.com/score', 'openapi_uri': 'https://xgboost-model-endpoint-2e60ced8.ukwest.inference.ml.azure.com/swagger.json', 'name': 'xgboost-model-endpoint-2e60ced8', 'description': "this is an online endpoint for the xgboost model and it's great!", 'tags': {'training_dataset': 'sklearn_iris'}, 'properties': {'createdBy': 'Ben Dixon', 'createdAt': '2024-05-24T15:47:06.025023+0000', 'lastModifiedAt': '2024-05-24T15:47:06.025023+0000', 'azureml.onlineendpointid': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourcegroups/aiplatform-1/providers/microsoft.machinelearningservices/workspaces/ai-mlw-dev-01/onlineendpoints/xgboost-model-endpoint-2e60ced8', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/providers/Microsoft.MachineLearningServices/locations/ukwest/m

In [28]:
model = ml_client.models.get(name=REGISTERED_MODEL_NAME, version=latest_model_version)
#local_model = Model(path="../deploy/assets/xgboost_model.json")

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
    code_configuration=CodeConfiguration(
        code="../deploy/", 
        scoring_script="score.py"
    ),
    environment=env
)

blue_deployment = ml_client.online_deployments.begin_create_or_update(
        deployment=blue_deployment
).result()

Check: endpoint xgboost-model-endpoint-2e60ced8 exists


...........

In [29]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://xgboost-model-endpoint-2e60ced8.ukwest.inference.ml.azure.com/score', 'openapi_uri': 'https://xgboost-model-endpoint-2e60ced8.ukwest.inference.ml.azure.com/swagger.json', 'name': 'xgboost-model-endpoint-2e60ced8', 'description': "this is an online endpoint for the xgboost model and it's great!", 'tags': {'training_dataset': 'sklearn_iris'}, 'properties': {'createdBy': 'Ben Dixon', 'createdAt': '2024-05-24T15:47:06.025023+0000', 'lastModifiedAt': '2024-05-24T15:47:06.025023+0000', 'azureml.onlineendpointid': '/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/resourcegroups/aiplatform-1/providers/microsoft.machinelearningservices/workspaces/ai-mlw-dev-01/onlineendpoints/xgboost-model-endpoint-2e60ced8', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/3979595d-b401-4ff4-b3ef-661b8481c742/providers/Microsoft.MachineLearningServices/locations/ukwest/m

Boo-yah!

In [30]:
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="assets/xgboost-request.json",)

'[1, 0, 2, 1]'

Endpoint details

In [31]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://xgboost-model-endpoint-2e60ced8.ukwest.inference.ml.azure.com/score


In [32]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

'Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Pulling, Type: Normal, Time: 2024-05-24T15:55:38.968185Z, Message: Start pulling container image\nKind: Pod, Name: Downloading, Type: Normal, Time: 2024-05-24T15:55:39.59076Z, Message: Start downloading models\nKind: Pod, Name: Pulled, Type: Normal, Time: 2024-05-24T15:56:35.866708Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2024-05-24T15:56:35.866708Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2024-05-24T15:56:35.948254Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2024-05-24T15:56:36.012062Z, Message: Started container inference-server\nKind: Pod, Name: ContainerReady, Type: Normal, Time: 2024-05-24T15:56:58.198984251Z, Message: Container is ready\n\nContainer logs

In [33]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

................................................................